In [1]:
import librosa
import numpy as np
import tarfile
import os
import csv
import math
from sklearn.multiclass import OneVsRestClassifier
from sklearn import linear_model
from sklearn.svm import LinearSVC
from sklearn import cross_validation
import scipy
from sklearn import preprocessing
from sklearn.decomposition import PCA
import dill
# Build a dictionary of file names
import csv

/opt/conda/lib/python2.7/site-packages/librosa/core/audio.py:37: UserWarning: Could not import scikits.samplerate. Falling back to scipy.signal
  warnings.warn('Could not import scikits.samplerate. '


In [2]:
with open('datasets/music_train_labels.csv', mode='r') as infile:
    reader = csv.reader(infile)
    fileGenreDict = {'data/train/'+rows[0]:rows[1] for rows in reader}
# Just as an example....To write the file back out .....    
#    with open('coors_new.csv', mode='w') as outfile:
#        writer = csv.writer(outfile)
# Generate Label Encoding
genreList = []
genreDistinct = []
for key, value in fileGenreDict.iteritems():
    if value not in genreList:
        genreDistinct.append(value)
    genreList.append(value)    
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(genreDistinct)
transformedGenres = le.transform(genreList)
invTransfedGenres = le.inverse_transform(transformedGenres)

In [4]:

# Function to create features
def create_features():
    x = []
    mono, fs = librosa.load('datasets/processed/mp3File.mp3', sr = 44100)
    frameLength = int(len(mono)/20)
    zcr = librosa.feature.zero_crossing_rate(mono, frame_length=frameLength, hop_length=frameLength-512, center=True)
    rms = librosa.feature.rmse(mono, S=None, n_fft=frameLength, hop_length=frameLength-512)
#    print 'Number of ZCR columns:%s\nNumber of RMS columns:%s' % (len(zcr[0]), len(rms[0])) 
    
    rms_mean = np.mean(rms[0])
    rms_std  = np.std(rms[0])
    rms_skew = scipy.stats.skew(rms[0], axis=0, bias=True)
#    rms_kurt = scipy.stats.kurtosis(rms[0], axis=0, fisher=True, bias=True)
 
    zcr_mean = np.mean(zcr[0])
    zcr_std  = np.std(zcr[0])
    zcr_skew = scipy.stats.skew(zcr[0], axis=0, bias=True)
#    zcr_kurt = scipy.stats.kurtosis(zcr[0], axis=0, fisher=True, bias=True)
    
#    x = np.append(zcr[0], rms[0])    
    x = [rms_mean, rms_std, rms_skew, zcr_mean, zcr_std, zcr_skew]
    
    return x
# Create features, one file at a time and build the dataset for classification
t = tarfile.open("datasets/music_train.tar", 'r')
y = []
X = []

In [5]:
for f in t.getnames():
    t.extract(f, 'datasets/processed/')
    os.rename('datasets/processed/'+f, 'datasets/processed/mp3File.mp3')
    genre = fileGenreDict[f]
    y.append(le.transform(genre))
#    print y, genre
# --- Call the function to create the features using mp3File.mp3 ......    
    X.append(create_features())
t.close() 

In [8]:
print len(X)
print len(y)

1167
1167


In [10]:

X_train, X_test, y_train, y_test = cross_validation.train_test_split(\
                    X, y, test_size=0.2, random_state=42) 
q1Model = OneVsRestClassifier(LinearSVC(random_state=0)).fit(X_train, y_train)
q1Model.score(X_test, y_test)

0.5

In [11]:
# Apply the model generated above to the Grader data : music_feature_extraction_test.tar
# First Create features, one file at a time and build the test dataset for classification
t = tarfile.open("datasets/music_feature_extraction_test.tar", 'r')
yt = []
Xt = []
for f in t.getnames():
    t.extract(f, 'datasets/processed/')
    os.rename('datasets/processed/'+f, 'datasets/processed/mp3File.mp3')
    yt.append(f)
# --- Call the function to create the features using mp3File.mp3 ......    
    Xt.append(create_features())
t.close()

In [24]:
yt2 = [i[-16:] for i in yt]
print yt2
print yt2[144]

['fe_test_0001.mp3', 'fe_test_0002.mp3', 'fe_test_0003.mp3', 'fe_test_0004.mp3', 'fe_test_0005.mp3', 'fe_test_0006.mp3', 'fe_test_0007.mp3', 'fe_test_0008.mp3', 'fe_test_0009.mp3', 'fe_test_0010.mp3', 'fe_test_0011.mp3', 'fe_test_0012.mp3', 'fe_test_0013.mp3', 'fe_test_0014.mp3', 'fe_test_0015.mp3', 'fe_test_0016.mp3', 'fe_test_0017.mp3', 'fe_test_0018.mp3', 'fe_test_0019.mp3', 'fe_test_0020.mp3', 'fe_test_0021.mp3', 'fe_test_0022.mp3', 'fe_test_0023.mp3', 'fe_test_0024.mp3', 'fe_test_0025.mp3', 'fe_test_0026.mp3', 'fe_test_0027.mp3', 'fe_test_0028.mp3', 'fe_test_0029.mp3', 'fe_test_0030.mp3', 'fe_test_0031.mp3', 'fe_test_0032.mp3', 'fe_test_0033.mp3', 'fe_test_0034.mp3', 'fe_test_0035.mp3', 'fe_test_0036.mp3', 'fe_test_0037.mp3', 'fe_test_0038.mp3', 'fe_test_0039.mp3', 'fe_test_0040.mp3', 'fe_test_0041.mp3', 'fe_test_0042.mp3', 'fe_test_0043.mp3', 'fe_test_0044.mp3', 'fe_test_0045.mp3', 'fe_test_0046.mp3', 'fe_test_0047.mp3', 'fe_test_0048.mp3', 'fe_test_0049.mp3', 'fe_test_0050.mp3',

In [32]:

outPredict = le.inverse_transform(q1Model.predict(Xt))
# Q1 Output
#for i in xrange(len(outPredict)):
#    print yt2[i]+':'+ outPredict[i]

In [30]:
q1_ans = dict((yt2[i], outPredict[i]) for i in range(0,145))
print len(q1_ans)
print q1_ans

145
{'fe_test_0041.mp3': 'rock', 'fe_test_0023.mp3': 'rock', 'fe_test_0079.mp3': 'jazz', 'fe_test_0144.mp3': 'raphiphop', 'fe_test_0133.mp3': 'raphiphop', 'fe_test_0019.mp3': 'jazz', 'fe_test_0052.mp3': 'jazz', 'fe_test_0059.mp3': 'raphiphop', 'fe_test_0017.mp3': 'jazz', 'fe_test_0103.mp3': 'rock', 'fe_test_0111.mp3': 'raphiphop', 'fe_test_0024.mp3': 'jazz', 'fe_test_0012.mp3': 'rock', 'fe_test_0075.mp3': 'rock', 'fe_test_0115.mp3': 'jazz', 'fe_test_0062.mp3': 'jazz', 'fe_test_0137.mp3': 'rock', 'fe_test_0055.mp3': 'jazz', 'fe_test_0093.mp3': 'raphiphop', 'fe_test_0049.mp3': 'jazz', 'fe_test_0016.mp3': 'rock', 'fe_test_0026.mp3': 'rock', 'fe_test_0002.mp3': 'rock', 'fe_test_0065.mp3': 'raphiphop', 'fe_test_0129.mp3': 'jazz', 'fe_test_0045.mp3': 'rock', 'fe_test_0057.mp3': 'rock', 'fe_test_0127.mp3': 'jazz', 'fe_test_0015.mp3': 'raphiphop', 'fe_test_0124.mp3': 'rock', 'fe_test_0114.mp3': 'rock', 'fe_test_0145.mp3': 'raphiphop', 'fe_test_0120.mp3': 'raphiphop', 'fe_test_0089.mp3': 'raphi

In [31]:
############
############
# Question 2


In [33]:
def create_features2():
    x = []
    mono, fs = librosa.load('datasets/processed/mp3File.mp3', sr = 44100)
    frameLength = int(len(mono)/20)
    zcr = librosa.feature.zero_crossing_rate(mono, frame_length=2048, hop_length=512, center=True)
    rms = librosa.feature.rmse(mono, S=None, n_fft=2048, hop_length=512)
    mfcc= librosa.feature.mfcc(mono, sr=44100, S=None, n_mfcc=24)
    # And the first-order differences (delta features)
    mfcc_delta = librosa.feature.delta(mfcc)
#    print 'Number of ZCR columns:%s\nNumber of RMS columns:%s' % (len(zcr[0]), len(rms[0])) 
#    zcr:  (1, 867)
#    rms:  (1, 867)
#    mfcc:  (20, 867)
    
    rms_mean = np.mean(rms[0])
    rms_std  = np.std(rms[0])
    rms_skew = scipy.stats.skew(rms[0], axis=0, bias=True)
#    rms_kurt = scipy.stats.kurtosis(rms[0], axis=0, fisher=True, bias=True)
            
    zcr_mean = np.mean(zcr[0])
    zcr_std  = np.std(zcr[0])
    zcr_skew = scipy.stats.skew(zcr[0], axis=0, bias=True)
#    zcr_kurt = scipy.stats.kurtosis(zcr[0], axis=0, fisher=True, bias=True)
  
    mfcc_mean = []
    mfcc_std  = []
    mfcc_skew = []
    mfcc_kurt = []
    for i in range(len(mfcc[:, 0])):
        mfcc_mean.append(np.mean(mfcc[i]))
        mfcc_std.append(np.std(mfcc[i]))
        mfcc_skew.append(scipy.stats.skew(mfcc[i], axis=0, bias=True))
        mfcc_kurt.append(scipy.stats.kurtosis(mfcc[i], axis=0, fisher=True, bias=True))
    mfcc_delta_mean = []
    mfcc_delta_std  = []
    mfcc_delta_skew = []
    mfcc_delta_kurt = []
    for i in range(len(mfcc_delta[:, 0])):
        mfcc_delta_mean.append(np.mean(mfcc_delta[i]))
        mfcc_delta_std.append(np.std(mfcc_delta[i]))
        mfcc_delta_skew.append(scipy.stats.skew(mfcc_delta[i], axis=0, bias=True))
        mfcc_delta_kurt.append(scipy.stats.kurtosis(mfcc_delta[i], axis=0, fisher=True, bias=True))
        
    # Separate harmonics and percussives into two waveforms
#    y_harmonic, y_percussive = librosa.effects.hpss(mono)
    # Beat track on the percussive signal
#    tempo, beat_frames = librosa.beat.beat_track(y=y_percussive, sr=44100)
    
        
    x = [[rms_mean, rms_std, rms_skew, zcr_mean, zcr_std, zcr_skew],\
          mfcc_mean, mfcc_std, mfcc_skew, mfcc_kurt,\
          mfcc_delta_mean, mfcc_delta_std, mfcc_delta_skew, mfcc_delta_kurt]
    
    X = [item for sublist in x for item in sublist]
    
    return X

In [34]:

# Create features, one file at a time and build the dataset for classification
t = tarfile.open("datasets/music_train.tar", 'r')
y = []
X = []
for f in t.getnames():
    t.extract(f, 'datasets/processed/')
    os.rename('datasets/processed/'+f, 'datasets/processed/mp3File.mp3')
    genre = fileGenreDict[f]
    y.append(le.transform(genre))
#    print y, genre
# --- Call the function to create the features using mp3File.mp3 ......    
    X.append(create_features2())
t.close() 

In [35]:
scaler = preprocessing.StandardScaler().fit(X)
XScaled = scaler.transform(X)
le.inverse_transform(y)
# Standardize the X data
scaler = preprocessing.StandardScaler().fit(X)
# Scale X before applying PCA
X_Scaled = scaler.transform(X)
pca = PCA(n_components=50)
q2pca = pca.fit(X_Scaled)
X_train, X_test, y_train, y_test = cross_validation.train_test_split(\
                    X_Scaled, y, test_size=0.2, random_state=0) 
# Reshape the X and y vectors
X_almost_ready = np.asarray(X_train).reshape(len(X_train),198).astype(np.float)
y_ready = np.asarray(y_train).reshape(len(y_train),1)
# Apply the PCA transformer
X_ready = q2pca.transform(X_almost_ready)
# Build the model
q2Model = OneVsRestClassifier(LinearSVC(random_state=0)).fit(X_ready, y_ready)
q2Model.score(X_ready, y_ready)

0.75026795284030012

In [36]:
# Try out the model on the holdout dataset
##################################################################################################
# Scale X before applying the model
#X_test_Scaled = scaler.transform(X_test)
X_test_Scaled = X_test
# Reshape the X vector
X_test_almost_ready = np.asarray(X_test_Scaled).reshape(len(X_test_Scaled),198).astype(np.float)
# Apply the PCA transformer
X_test_ready = q2pca.transform(X_test_almost_ready)
# Apply the model
y_pred = q2Model.predict(X_test_ready)
# Convert the encoded labels back
y_pred_decode = le.inverse_transform(y_pred)
y_test_ready = np.asarray(y_test).reshape(len(y_test),1)
q2Model.score(X_test_ready, y_test_ready)

0.70940170940170943

In [37]:

# Apply the model generated above to the Grader data : music_feature_extraction_test.tar

# First Create features, one file at a time and build the test dataset for classification
t = tarfile.open("datasets/music_feature_extraction_test.tar", 'r')
yt = []
Xt = []
for f in t.getnames():
    t.extract(f, 'datasets/processed/')
    os.rename('datasets/processed/'+f, 'datasets/processed/mp3File.mp3')
    yt.append(f)
# --- Call the function to create the features using mp3File.mp3 ......    
    Xt.append(create_features2())
t.close()
# Scale X before fitting
Xt_Scaled = scaler.transform(Xt)
# Reshape the X vector
Xt_almost_ready = np.asarray(Xt_Scaled).reshape(len(Xt_Scaled),198).astype(np.float)
# Apply the PCA transformer
Xt_ready = q2pca.transform(Xt_almost_ready)
# Apply the model
yt_pred = q2Model.predict(Xt_ready)
outPredict = le.inverse_transform(q2Model.predict(Xt_ready))
# Q2 Output
for i in xrange(len(outPredict)):
    print yt[i][29:], outPredict[i]

fe_test_0001.mp3 jazz
fe_test_0002.mp3 rock
fe_test_0003.mp3 raphiphop
fe_test_0004.mp3 rock
fe_test_0005.mp3 rock
fe_test_0006.mp3 rock
fe_test_0007.mp3 rock
fe_test_0008.mp3 rock
fe_test_0009.mp3 rock
fe_test_0010.mp3 raphiphop
fe_test_0011.mp3 jazz
fe_test_0012.mp3 rock
fe_test_0013.mp3 rock
fe_test_0014.mp3 rock
fe_test_0015.mp3 rock
fe_test_0016.mp3 rock
fe_test_0017.mp3 raphiphop
fe_test_0018.mp3 raphiphop
fe_test_0019.mp3 jazz
fe_test_0020.mp3 rock
fe_test_0021.mp3 jazz
fe_test_0022.mp3 rock
fe_test_0023.mp3 rock
fe_test_0024.mp3 jazz
fe_test_0025.mp3 raphiphop
fe_test_0026.mp3 folkcountry
fe_test_0027.mp3 rock
fe_test_0028.mp3 rock
fe_test_0029.mp3 rock
fe_test_0030.mp3 rock
fe_test_0031.mp3 rock
fe_test_0032.mp3 jazz
fe_test_0033.mp3 raphiphop
fe_test_0034.mp3 jazz
fe_test_0035.mp3 rock
fe_test_0036.mp3 rock
fe_test_0037.mp3 jazz
fe_test_0038.mp3 folkcountry
fe_test_0039.mp3 jazz
fe_test_0040.mp3 rock
fe_test_0041.mp3 rock
fe_test_0042.mp3 rock
fe_test_0043.mp3 rock
fe_test_00

In [38]:
yt2 = [i[-16:] for i in yt]
print yt2
print yt2[144]

['fe_test_0001.mp3', 'fe_test_0002.mp3', 'fe_test_0003.mp3', 'fe_test_0004.mp3', 'fe_test_0005.mp3', 'fe_test_0006.mp3', 'fe_test_0007.mp3', 'fe_test_0008.mp3', 'fe_test_0009.mp3', 'fe_test_0010.mp3', 'fe_test_0011.mp3', 'fe_test_0012.mp3', 'fe_test_0013.mp3', 'fe_test_0014.mp3', 'fe_test_0015.mp3', 'fe_test_0016.mp3', 'fe_test_0017.mp3', 'fe_test_0018.mp3', 'fe_test_0019.mp3', 'fe_test_0020.mp3', 'fe_test_0021.mp3', 'fe_test_0022.mp3', 'fe_test_0023.mp3', 'fe_test_0024.mp3', 'fe_test_0025.mp3', 'fe_test_0026.mp3', 'fe_test_0027.mp3', 'fe_test_0028.mp3', 'fe_test_0029.mp3', 'fe_test_0030.mp3', 'fe_test_0031.mp3', 'fe_test_0032.mp3', 'fe_test_0033.mp3', 'fe_test_0034.mp3', 'fe_test_0035.mp3', 'fe_test_0036.mp3', 'fe_test_0037.mp3', 'fe_test_0038.mp3', 'fe_test_0039.mp3', 'fe_test_0040.mp3', 'fe_test_0041.mp3', 'fe_test_0042.mp3', 'fe_test_0043.mp3', 'fe_test_0044.mp3', 'fe_test_0045.mp3', 'fe_test_0046.mp3', 'fe_test_0047.mp3', 'fe_test_0048.mp3', 'fe_test_0049.mp3', 'fe_test_0050.mp3',

In [39]:
q2_ans = dict((yt2[i], outPredict[i]) for i in range(0,145))
print len(q2_ans)
print q2_ans

145
{'fe_test_0041.mp3': 'rock', 'fe_test_0023.mp3': 'rock', 'fe_test_0079.mp3': 'rock', 'fe_test_0144.mp3': 'folkcountry', 'fe_test_0133.mp3': 'raphiphop', 'fe_test_0019.mp3': 'jazz', 'fe_test_0052.mp3': 'jazz', 'fe_test_0059.mp3': 'raphiphop', 'fe_test_0017.mp3': 'raphiphop', 'fe_test_0103.mp3': 'rock', 'fe_test_0111.mp3': 'jazz', 'fe_test_0024.mp3': 'jazz', 'fe_test_0012.mp3': 'rock', 'fe_test_0075.mp3': 'rock', 'fe_test_0115.mp3': 'folkcountry', 'fe_test_0062.mp3': 'folkcountry', 'fe_test_0137.mp3': 'folkcountry', 'fe_test_0055.mp3': 'folkcountry', 'fe_test_0093.mp3': 'raphiphop', 'fe_test_0049.mp3': 'jazz', 'fe_test_0016.mp3': 'rock', 'fe_test_0026.mp3': 'folkcountry', 'fe_test_0002.mp3': 'rock', 'fe_test_0065.mp3': 'raphiphop', 'fe_test_0129.mp3': 'folkcountry', 'fe_test_0045.mp3': 'electronic', 'fe_test_0057.mp3': 'jazz', 'fe_test_0127.mp3': 'raphiphop', 'fe_test_0015.mp3': 'rock', 'fe_test_0124.mp3': 'raphiphop', 'fe_test_0114.mp3': 'folkcountry', 'fe_test_0145.mp3': 'raphiphop

In [41]:
##### Q3


In [40]:
print 'a'

a


In [42]:
# Q3
import librosa
import numpy as np
import tarfile
import os
import csv
import math
from sklearn.multiclass import OneVsRestClassifier
from sklearn import linear_model
from sklearn.svm import LinearSVC
from sklearn import cross_validation
import scipy
from sklearn import preprocessing
from sklearn.decomposition import PCA
import dill
df_train_list = []
with open('datasets/df_train_anon.csv', 'rb') as f:
    reader = csv.reader(f)
    df_train_list = list(reader)

In [47]:
len(df_train_list[0])
yq3 = []
Xq3 = []
temp_list = []
for rows in df_train_list:
    temp_list.append(rows)       
    yq3.append(temp_list[0][-1])
    Xq3.append(temp_list[0][0:-1])
    temp_list = []

In [50]:
print len(yq3)
print len(Xq3)
print yq3[:5]

1167
1167
['jazz', 'jazz', 'electronic', 'jazz', 'electronic']


In [59]:
# Standardize the X data
scaler = preprocessing.StandardScaler().fit(Xq3)
#print Xq3[0]
#print scaler
# Generate Label Encoding
le = preprocessing.LabelEncoder().fit(yq3)
# le.classes_
# invTransf_y = le.inverse_transform(y)
# Scale X before applying PCA
Xq3Scaled = scaler.transform(Xq3)
# Build PCA transformer
pca = PCA(n_components=80)
q3pca = pca.fit(Xq3Scaled)

In [61]:
#Xfit_tran = pca.fit_transform(X_train)
X_train=Xq3
y_train=yq3
# Scale X and transform y before building model
X_train_Scaled = scaler.transform(X_train)
y_train_le = le.transform(y_train)
# Reshape the X and y vectors
X_almost_ready = np.asarray(X_train_Scaled).reshape(len(X_train_Scaled),549).astype(np.float)
y_ready = np.asarray(y_train_le).reshape(len(y_train_le),1)
# Apply the PCA transformer
X_ready = q3pca.transform(X_almost_ready)

In [70]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(\
                    X_ready, y_ready, test_size=0.2, random_state=0)

# Build the model
q3Model = OneVsRestClassifier(LinearSVC(random_state=0)).fit(X_train, y_train)
q3Model.score(X_test, y_test)

0.76068376068376065

In [73]:
# for the grader
#rec = np.asarray(record).reshape(1,549).astype(np.float)
#rec_transformed = q3pca.transform(rec)
#yPred = q3Model.predict(rec_transformed)
#le.inverse_transform(yPred[0])
dill.dump(q3Model, open('Model.dill', 'w'))
dill.dump(q3pca, open('PCA.dill', 'w'))
dill.dump(le, open('Labels.dill', 'w'))
dill.dump(scaler, open('Scaler.dill','w'))

In [ ]:
### other attempted models for Q2
# Function to create features
def create_features():
    x = []
    mono, fs = librosa.load('datasets/processed/mp3File.mp3', sr = 44100)
    frameLength = int(len(mono)/20)
    zcr = librosa.feature.zero_crossing_rate(mono, frame_length=2048, hop_length=512, center=True)
    rms = librosa.feature.rmse(mono, S=None, n_fft=2048, hop_length=512)
    mfcc= librosa.feature.mfcc(mono, sr=44100, S=None, n_mfcc=20)
    spec_bw = librosa.feature.spectral_bandwidth(y=mono, sr=44100)
    chroma = librosa.feature.chroma_stft(mono, sr=44100, S=None, norm=np.inf, n_fft=2048, hop_length=512, tuning=None)
    mel_spec = librosa.feature.melspectrogram(mono, sr=44100, S=None, n_fft=2048, hop_length=512)
    spec_cent = librosa.feature.spectral_centroid(mono, sr=44100, S=None, n_fft=2048, hop_length=512, freq=None)
    spec_band = librosa.feature.spectral_bandwidth(mono, sr=44100, S=None, n_fft=2048, hop_length=512, freq=None, centroid=None,\
                                                   norm=True, p=2)
    spec_contr = librosa.feature.spectral_contrast(mono, sr=44100, S=None, n_fft=2048, hop_length=512, freq=None, fmin=200.0,\
                                                   n_bands=6, quantile=0.02, linear=False)
    spec_rolloff = librosa.feature.spectral_rolloff(mono, sr=44100, S=None, n_fft=2048, hop_length=512, freq=None, roll_percent=0.85)
    tonnetz = librosa.feature.tonnetz(mono, sr=44100, chroma=None)
    tempogram = librosa.feature.tempogram(mono, sr=44100, onset_envelope=None, hop_length=512, win_length=384, center=True,\
                                          window=None, norm=np.inf)
#    print 'Number of ZCR columns:%s\nNumber of RMS columns:%s' % (len(zcr[0]), len(rms[0])) 
#    zcr:  (1, 867)
#    rms:  (1, 867)
#    mfcc:  (20, 867)
#    spec_bw:  (1, 867)
#    chroma:  (12, 867)
#    mel_spec:  (128, 867)
#    spec_cent:  (1, 867)
#    spec_band:  (1, 867)
#    spec_contr:  (7, 867)
#    spec_rolloff:  (1, 867)
#    tonnetz:  (6, 867)
#    tempogram:  (384, 867)
    
    rms_mean = np.mean(rms[0])
    rms_std  = np.std(rms[0])
    rms_skew = scipy.stats.skew(rms[0], axis=0, bias=True)
 
    zcr_mean = np.mean(zcr[0])
    zcr_std  = np.std(zcr[0])
    zcr_skew = scipy.stats.skew(zcr[0], axis=0, bias=True)
    
    spec_cent_mean = np.mean(spec_cent[0])
    spec_cent_std  = np.std(spec_cent[0])
    spec_cent_skew = scipy.stats.skew(spec_cent[0], axis=0, bias=True)
   
    spec_band_mean = np.mean(spec_band[0])
    spec_band_std  = np.std(spec_band[0])
    spec_band_skew = scipy.stats.skew(spec_band[0], axis=0, bias=True)
       
    spec_rolloff_mean = np.mean(spec_rolloff[0])
    spec_rolloff_std  = np.std(spec_rolloff[0])
    spec_rolloff_skew = scipy.stats.skew(spec_rolloff[0], axis=0, bias=True)
       
    spec_bw_mean = np.mean(spec_bw[0])
    spec_bw_std  = np.std(spec_bw[0])
    spec_bw_skew = scipy.stats.skew(spec_bw[0], axis=0, bias=True)
       
    mfcc_mean = []
    mfcc_std  = []
    mfcc_skew = []
    mfcc_kurt = []
    for i in range(len(mfcc[:, 0])):
        mfcc_mean.append(np.mean(mfcc[i]))
        mfcc_std.append(np.std(mfcc[i]))
        mfcc_skew.append(scipy.stats.skew(mfcc[i], axis=0, bias=True))
        mfcc_kurt.append(scipy.stats.kurtosis(mfcc[i], axis=0, fisher=True, bias=True))
        
    
    chroma_mean = []
    chroma_std  = []
    chroma_skew = []
    for i in range(len(chroma[:, 0])):
        chroma_mean.append(np.mean(chroma[i,0]))
        chroma_std.append(np.std(chroma[i]))
        chroma_skew.append(scipy.stats.skew(chroma[i], axis=0, bias=True))
        
    mel_spec_mean = []
    mel_spec_std  = []
    mel_spec_skew = []
    for i in range(len(mel_spec[:, 0])):
        mel_spec_mean.append(np.mean(mel_spec[i]))
        mel_spec_std.append(np.std(mel_spec[i]))
        mel_spec_skew.append(scipy.stats.skew(mel_spec[i], axis=0, bias=True))
        
    spec_contr_mean = []
    spec_contr_std  = []
    spec_contr_skew = []
    for i in range(len(spec_contr[:, 0])):
        spec_contr_mean.append(np.mean(spec_contr[i]))
        spec_contr_std.append(np.std(spec_contr[i]))
        spec_contr_skew.append(scipy.stats.skew(spec_contr[i], axis=0, bias=True))
        
    tonnetz_mean = []
    tonnetz_std  = []
    tonnetz_skew = []
    for i in range(len(tonnetz[:, 0])):
        tonnetz_mean.append(np.mean(tonnetz[i]))
        tonnetz_std.append(np.std(tonnetz[i]))
        tonnetz_skew.append(scipy.stats.skew(tonnetz[i], axis=0, bias=True))
        
    tempogram_mean = []
    tempogram_std  = []
    tempogram_skew = []
    for i in range(len(tempogram[:, 0])):
        tempogram_mean.append(np.mean(tempogram[i]))                
        tempogram_std.append(np.std(tempogram[i]))
        tempogram_skew.append(scipy.stats.skew(tempogram[i], axis=0, bias=True))
        
    x = [[rms_mean, rms_std, rms_skew, zcr_mean, zcr_std, zcr_skew, spec_bw_mean, spec_bw_std, spec_bw_skew],\
         [spec_cent_mean, spec_cent_std, spec_cent_skew, spec_band_mean, spec_band_std, spec_band_skew],\
         [spec_rolloff_mean, spec_rolloff_std, spec_rolloff_skew],\
         mfcc_mean, mfcc_std, mfcc_skew, chroma_mean, chroma_std, chroma_skew, mel_spec_mean, mel_spec_std, mel_spec_skew,\
         spec_contr_mean, spec_contr_std, spec_contr_skew, tonnetz_mean, tonnetz_std, tonnetz_skew,\
         tempogram_mean, tempogram_std, tempogram_skew ]
    
    X = [item for sublist in x for item in sublist]
    
    return X